## Import the pipeline functions from functions.py

In [24]:
%run -i functions_edit.py

# USER INPUT - **please enter the parameters of your experiment:**

In [4]:
# Default is channels=['CY5', 'CY3', 'CY3.5', 'DAPI']
channels = ['CY5', 'CY3.5', 'CY3', 'DAPI']

# Number of Z-slices per channel (for Marah's data this is 41)
number_zslices = 41

# Default is colors=['magenta', 'green', 'cyan', 'blue']
colors = ['magenta', 'green', 'cyan', 'blue']

# Defaults is voxel_size=(200, 64.5, 64.5) in nanometers
voxel_size=(200, 64.5, 64.5)

# Default is spot_radius=(200, 70, 70)
spot_radius=(200, 70, 70)

#image scale in micrometres
image_scale = (voxel_size[1]/1000, voxel_size[2]/1000)


# Please add the paths to the three Cellpose models downloaded from the repo.
pretrained_whole_model_path = "C:/Users/lotta/Pictures/Cellpose/whole_DAPI+DIC/models/whole_model_993"
# If you do not wish to use the Cellpose models below set their value to None as follows: 
# pretrained_separate_model_path = None
# pretrained_whole_SHIFTCORRECTED_model_path = None
pretrained_separate_model_path = "C:/Users/lotta/Pictures/Cellpose/sep_DAPI+DIC/models/sep_model_1180"
pretrained_whole_SHIFTCORRECTED_model_path = None

yET916_BR2_03 = ['C:/Users/lotta/Documents/Bioinformatics_Msc/Project/DATA_INITIAL/pair_1/yET916-SUN4Q570-SRL1CFR610-ASH1CLB2Q670_03_CY5, CY3.5 NAR, CY3, DAPI.tif',
                 "C:/Users/lotta/Documents/Bioinformatics_Msc/Project/DATA_INITIAL/pair_1/yET916-SUN4Q570-SRL1CFR610-ASH1CLB2Q670_04_DIC-100.tif"]

# Please add the path of the TIF and DIC file you'd like to analyse with this pipeline
tif_path = yET916_BR2_03[0]
dic_path = yET916_BR2_03[1]

# Please add the path to which you'd like to save the Dataframe containing the metrics calculates
results_excel_path = 'C:/Users/lotta/Bioinformatics/smFISHimagepipeline/pipeline_outputs/output_yET916_BR2_03.xlsx'

#### Read corresponding DIC and TIF images and split the TIF file into the four channels
- "corresponding_files" is a list of the corresponding file names. 
- "stack_example" is an array of the 41 z-slices for each of the 4 channels.

### Load TIF and DIC image

In [5]:
corresponding_files, image_stack = read_stack([tif_path, dic_path], number_zslices, channels)

100%|████████████████████████████████████████████████████████████████████████████████| 164/164 [00:02<00:00, 80.28it/s]


#### For each channel create a Maximum Intensity Projection for each channel
- For each channel there are 40 slices of these not all are in focus, if the "in-focus-zslices" are known they can be input into the function. 
- If not a laplacian operator can be used to select these "in-focus-zslices".

In [6]:
# Create Maximum Intensity Projections for each channel
# Each MIP image array is iteratively added to the projection list.
projection = []
for i in image_stack:
    # Instead of choose_focus_lap(i) the focussed slices can be 
    # input in the form [first in focus slice, last in focus slice]
    infocus_slices = choosing_in_focus_slices(i, number_of_slices=12)
    projection.append(generating_maximum_projection(i, infocus_slices))

#### View the MIPs alongside the DIC in napari

In [7]:
# View the MIPs in napari.
viewer= napari.Viewer()
napari_view(corresponding_files, np.array(projection), channels, colors, image_scale)
viewer.scale_bar.visible = True
viewer.scale_bar.unit = "um"

## Detect the mRNA spots for each of the zslices in each channel
- "spot_coord_dict" is a dictionary where the key is each channel and the value is an array of the mRNA coordinates.

In [8]:
# RNA detection for each channel.
spot_coord_dict = detecting_mRNA_spots(image_stack, voxel_size, spot_radius, channels)

In [11]:
spot_dict_df = saving_spot_dict_in_df(spot_coord_dict)

#### View the MIPs alongside the DIC with the detected spots in napari

In [12]:
viewer= napari.Viewer()
napari_view(corresponding_files, np.array(projection), channels, colors, image_scale)
napari_view_spots(corresponding_files, np.array(projection), spot_coord_dict, channels, colors, image_scale)
viewer.scale_bar.visible = True
viewer.scale_bar.unit = "um"

# Cellpose
#### Load pre-trained models
- These can be substituted with any user-trained model and there is no need to use two, this is simply for mother-bud analysis. 

In [15]:
whole_model = models.CellposeModel(pretrained_model=pretrained_whole_model_path)
if pretrained_separate_model_path != None:
    sep_model = models.CellposeModel(pretrained_model=pretrained_separate_model_path)

#### Create a two channel image with the DAPI MIP and the DIC image to generate Cellpose masks

In [16]:
channel_dictionary = split_stack(image_stack, channels, colors)
in_focus_slices = choosing_in_focus_slices(channel_dictionary['DAPI'][1], number_of_slices=15)
dapi_projection = (generating_maximum_projection(channel_dictionary['DAPI'][1], in_focus_slices))
cellpose_input = [io.imread(corresponding_files[1]), dapi_projection]

#### Choose images to create masks of using the model

In [17]:
imgs = np.array(cellpose_input)
nimg = len(imgs)

#### Evaluate the images and return masks

In [22]:
# define CHANNELS to run segementation on
# grayscale=0, R=1, G=2, B=3
# channels = [cytoplasm, nucleus]
# if NUCLEUS channel does not exist, set the second channel to 0
cellpose_channels = [[1,2]]
masks_whole, whole_flows, whole_styles = whole_model.eval(imgs, diameter=None, channels=cellpose_channels)
if pretrained_separate_model_path != None:
    masks_sep, sep_flows, sep_styles = sep_model.eval(imgs, diameter=None, channels=cellpose_channels)

# Oppurtunity to view and correct masks using napari
#### Using the labels layer in napari you are able to use paintbrush and eraser tools to edit the masks. 
- Use the dropper tool to match the colour of the mask you want to edit. 
- Or simply use any colour to draw a new one on. 
- Use the shuffle colours button to check that your edit, of i.e. addition to a mask, is recognised as part of the mask. 
    - If this is working correctly when you shuffle colours the edit and the old mask should change to the same colour as one. 
#### Once you have edited as desired please save the masks in a tif file
- This can be done within napari by going to --> save selected layers and then saving in your file system. 
- Note down where you have saved this image as you will be asked for a path of this to continue with the pipeline

In [25]:
napari_check_and_edit_masks(masks_whole, corresponding_files[1], "Whole")

In [26]:
if pretrained_separate_model_path != None:
    napari_check_and_edit_masks(masks_sep, corresponding_files[1], "Separate")

# USER INPUT - if you have edited the masks please put the corrected mask paths below:

In [27]:
corrected_whole_masks_path = "C:/Users/lotta/Pictures/corrected_cellpose_masks/whole_masks_corrected.tif"
corrected_sep_masks_path = "C:/Users/lotta/Pictures/corrected_cellpose_masks/sep_masks_corrected.tif"

#### Continuing with the pipeline:

In [28]:
masks_whole = loading_corrected_masks(masks_whole, corrected_whole_masks_path)

In [29]:
if pretrained_separate_model_path != None:
    masks_sep = loading_corrected_masks(masks_whole, corrected_sep_masks_path)

In [30]:
projection_dict = create_mip_projection_dict(channels, projection)

### Assign mRNA spots to individual cells
##### For the 'separate' masks i.e. the mother and the bud are two separate units

In [31]:
# Load masks of both Cellpose models.
spots_per_cell_whole_masks = extracting_spots_per_cell(spot_coord_dict, masks_whole, projection_dict)

In [ ]:
spots_per_cell_whole_masks

## Work out the DIC shift using the proportion of mRNA spots in cells vs outside of them 

In [32]:
proportion_dict = {}

In [33]:
# Default is channels=['rna_coord', 'CY3', 'CY3.5']
spot_channels = list(spot_coord_dict.keys())
spot_channels[0] = 'rna_coord'

In [34]:
prop, cells, total = counting_spots_in_cells(spot_coord_dict, spots_per_cell_whole_masks, spot_channels)
print(f'Original proportion of mRNA spots per cell: {prop}')
dic_shift_info = finding_max_proportion_of_image(prop, 0, 0, proportion_dict, spot_coord_dict, masks_whole, projection_dict)

Original proportion of mRNA spots per cell: 78.32151111819559
Current maximum proportion 82.13 and the cooresponding coordinates: [-1, 1]
Current maximum proportion 84.96 and the cooresponding coordinates: [-2, 2]
Current maximum proportion 87.66 and the cooresponding coordinates: [-3, 3]
Current maximum proportion 89.82 and the cooresponding coordinates: [-4, 4]
Current maximum proportion 91.46 and the cooresponding coordinates: [-5, 5]
Current maximum proportion 92.71 and the cooresponding coordinates: [-6, 6]
Current maximum proportion 93.42 and the cooresponding coordinates: [-7, 7]
Current maximum proportion 93.61 and the cooresponding coordinates: [-8, 8]
Current maximum proportion 93.67 and the cooresponding coordinates: [-9, 8]
Current maximum proportion 93.69 and the cooresponding coordinates: [-10, 7]


In [35]:
dic_shift_info

[93.68773411970989, [-10, 7]]

# Cellpose part 2

In [36]:
if pretrained_whole_SHIFTCORRECTED_model_path != None:
    shiftcorrected_whole_model = models.CellposeModel(pretrained_model=pretrained_whole_SHIFTCORRECTED_model_path)
    dapi_projection_shifted = shift(dapi_projection,  [-dic_shift_info[1][0],-dic_shift_info[1][1]])
    cellpose_input = [io.imread(corresponding_files[1]), dapi_projection_shifted]
    #
    cellpose_imgs = np.array(cellpose_input)
    nimg = len(imgs)
    #
    shiftmodel_masks_whole, shiftmodel_whole_flows, shiftmodel_whole_styles = shiftcorrected_whole_model.eval(cellpose_imgs, diameter=None, channels=cellpose_channels)

### Per cell mRNA coordinates with the shift 

In [37]:
if pretrained_separate_model_path != None:
    mask_sep_shifted = shift(masks_sep, dic_shift_info[1])

In [38]:
if pretrained_whole_SHIFTCORRECTED_model_path != None:
    mask_whole_shifted = shift(shiftmodel_masks_whole, dic_shift_info[1])
else:
    mask_whole_shifted = shift(masks_whole, dic_shift_info[1])

In [39]:
spots_per_cell_whole_shifted = extracting_spots_per_cell(spot_coord_dict, mask_whole_shifted, projection_dict)
if pretrained_separate_model_path != None:
    spots_per_cell_sep_shifted = extracting_spots_per_cell(spot_coord_dict, mask_sep_shifted, projection_dict)

### Find the corresponding mother and bud segments

In [40]:
# Assign the mother and bud fragments to eachother using the centre points of the masks.
if pretrained_separate_model_path != None:
    centroid_dict = mask_centroids(mapping_mask_coordinates(spots_per_cell_sep_shifted))
    motherbud_pairs = mother_bud_reunion(mapping_mask_coordinates(spots_per_cell_whole_shifted), centroid_dict)
    motherbud_dict = mother_or_bud(motherbud_pairs, spots_per_cell_sep_shifted) # fov index : mother or bud
    print(motherbud_dict)
else:
    motherbud_dict = None

{13: ('bud', 'corresponding mother: 15'), 15: ('mother', 'corresponding bud: 13'), 16: ('mother', 'corresponding bud: 19'), 19: ('bud', 'corresponding mother: 16'), 26: ('mother', 'corresponding bud: 30'), 30: ('bud', 'corresponding mother: 26'), 28: ('bud', 'corresponding mother: 31'), 31: ('mother', 'corresponding bud: 28'), 33: ('mother', 'corresponding bud: 37'), 37: ('bud', 'corresponding mother: 33'), 38: ('bud', 'corresponding mother: 41'), 41: ('mother', 'corresponding bud: 38'), 45: ('bud', 'corresponding mother: 47'), 47: ('mother', 'corresponding bud: 45'), 49: ('bud', 'corresponding mother: 51'), 51: ('mother', 'corresponding bud: 49'), 54: ('mother', 'corresponding bud: 62'), 62: ('bud', 'corresponding mother: 54'), 56: ('mother', 'corresponding bud: 60'), 60: ('bud', 'corresponding mother: 56'), 59: ('mother', 'corresponding bud: 70'), 70: ('bud', 'corresponding mother: 59'), 76: ('mother', 'corresponding bud: 85'), 85: ('bud', 'corresponding mother: 76'), 83: ('bud', 'co

#### View shifted masks with spots on MIPs with the corresponding DIC

In [42]:
viewer = napari.Viewer()
napari_view(corresponding_files, np.array(projection), channels, colors, image_scale)
napari_view_spots(corresponding_files, np.array(projection), spot_coord_dict, channels, colors, image_scale)
napari_view_masks(mapping_mask_coordinates(spots_per_cell_whole_shifted),  image_scale, "Whole Masks", mask_colors=['pink'])
if pretrained_separate_model_path != None:
    napari_view_masks(mapping_mask_coordinates(spots_per_cell_sep_shifted), image_scale, "Separate Masks", mask_colors=['lightblue'])
viewer.scale_bar.visible = True
viewer.scale_bar.unit = "um"

# Put all the information in a DataFrame

In [43]:
%run -i bigfish_function.py

In [44]:
spot_im_channels = list(projection_dict.keys())
spot_im_channels.remove('DAPIP')

In [45]:
if pretrained_separate_model_path != None:
    sep_df = writing_metrics_to_DataFrame(spots_per_cell_sep_shifted, spot_channels, spot_im_channels, motherbud_dict, image_scale, dic_shift_info)
else:
    sep_df = None
whole_df = writing_metrics_to_DataFrame(spots_per_cell_whole_shifted, spot_channels, spot_im_channels, None, image_scale, dic_shift_info)

## Save DataFrame as an Excel File

In [46]:
writing_dataframe_to_excel(results_excel_path, whole_df, sep_df, spot_dict_df)